In [ ]:
# Preparando o ambiente no Google Drive:

# Importando a biblioteca
from google.colab import drive

# Isso irá pedir sua autorização
drive.mount('/content/drive')

# Agora, seu Drive estará disponível em: /content/drive/My Drive

# Mapas Emocionais


## Importação das bibliotecas


In [ ]:
import sys
# Biblioteca para mapa
import folium
from folium import plugins

# Biblioteca para conexao com o banco de dados
import psycopg2
psycopg2.extensions.register_type(psycopg2.extensions.UNICODE)
psycopg2.extensions.register_type(psycopg2.extensions.UNICODEARRAY)

# Biblioteca para os dados em geojson
import json

# Bibliotecas dos widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

# Biblioteca de mediacao dos parametros de conexao do banco de dados
from configparser import ConfigParser


## Conexão com o banco de dados

In [ ]:
def config(filename='/content/drive/My Drive/Colab Notebooks/database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [ ]:
conn = None
try:
  # read connection parameters
  params = config()
  # connect to the PostgreSQL server
  print('Connecting to the PostgreSQL database...')
  conn = psycopg2.connect(**params)
  # create a cursor
  cursor = conn.cursor()
  print ('Conexao realizada com sucesso')
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  print ('Erro na conexao com o banco de dados')


Connecting to the PostgreSQL database...
Conexao realizada com sucesso


## Mapa 1 - Escolha uma emoção e veja em quais ruas ela foi atribuída



In [ ]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
# Combox com as emoções
# Funcao para fazer a lista do comboBox 
def lista_emoc():
  emoc = []
  cursor.execute("""SELECT cod_emoc_ruas, tipo FROM emocoes;""")
  fet_list = cursor.fetchall()
  conn.commit()
  for cod_emoc_ruas, tipo in fet_list:
    emoc.append((tipo, cod_emoc_ruas))
  return emoc

# Definindo a caixa de escolha
comboEmoc = widgets.Dropdown(
    options = lista_emoc(),
    description='Emocao:',
)
# Mostrando a caixa de escolha
display(comboEmoc)


In [ ]:
comboEmoc_esc = comboEmoc.value

# Consulta emocoes geometria linha
cursor.execute("""CREATE OR REPLACE VIEW emoc_selec AS SELECT ruas_emoc_wgs.* FROM ruas_emoc_wgs WHERE %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features 
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM emoc_selec As lg   ) As f )  As fc;""")
emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Consulta pontos sobre linhas
cursor.execute("""CREATE OR REPLACE VIEW emoc_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM emoc_selec; """)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM emoc_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Consulta emocoes geometria ponto
cursor.execute("""CREATE OR REPLACE VIEW pts_emoc_selec AS SELECT pts_emoc.*, ST_Transform(geom,4326) As geom_wgs FROM pts_emoc WHERE %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM pts_emoc_selec As lg   ) As f )  As fc;""")
pts_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()
# cursor.close()
# conn.close()

# Consulta emocoes por GENERO
# Feminino
cursor.execute("""CREATE OR REPLACE VIEW female_emoc_selec AS SELECT trj_user_modal.* FROM trj_user_modal, usuario 
WHERE trj_user_modal.user_id = usuario.user_id AND usuario.genero = 'feminino' AND %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT ogc_fid) As l)) As properties
FROM female_emoc_selec As lg ) As f )  As fc;""")
female_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Masculino
cursor.execute("""CREATE OR REPLACE VIEW male_emoc_selec AS SELECT trj_user_modal.* FROM trj_user_modal, usuario 
WHERE trj_user_modal.user_id = usuario.user_id AND usuario.genero = 'masculino' AND %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT ogc_fid) As l)) As properties
FROM male_emoc_selec As lg ) As f )  As fc;""")
male_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Definindo o mapa
m = folium.Map(location = [-25.44,-49.24],
               zoom_start = 15,
               min_zoom = 15,
               tiles = 'Cartodb Positron')

# Definindo um group layer para linhas mais pontos
ft_grp1 = folium.FeatureGroup(name = 'Emocoes linhas')

# Camada das linhas
ruas_emoc = folium.GeoJson(
    emoc_geojson[2:len(emoc_geojson)-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            },  
    overlay=True
    ).add_to(ft_grp1)

# Camada dos pontos sobre as linhas
gjson_lnstopts = json.loads(lnstopts_geojson[2:len(lnstopts_geojson)-2])

# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = '%s'; """ %comboEmoc_esc)
id_emoji = cursor.fetchall()
conn.commit()
icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  ft_grp1.add_child(marker)

# Definindo um group layer para os pontos
ft_grp2 = folium.FeatureGroup(name = 'Emocoes pontos')

# Camada dos pontos
gjson_marker = json.loads(pts_emoc_geojson[2:len(pts_emoc_geojson)-2])

# Estilizando os pontos do geojson
for feature in gjson_marker['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  ft_grp2.add_child(marker)

# Camada das emocoes por GENERO
# Definindo um group layer para linhas mais pontos - FEMININO
ft_grp3 = folium.FeatureGroup(name = 'Genero feminino', show = False)

# Camada das linhas - FEMININO
female_ruas_emoc = folium.GeoJson(
    female_emoc_geojson[2:len(female_emoc_geojson)-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'red',
            'weight': 2,
            },  
    overlay=True
    ).add_to(ft_grp3)

# Definindo um group layer para linhas mais pontos - MASCULINO
ft_grp4 = folium.FeatureGroup(name = 'Genero masculino', show = False)

# Camada das linhas - MASCULINO
male_ruas_emoc = folium.GeoJson(
    male_emoc_geojson[2:len(male_emoc_geojson)-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'blue',
            'weight': 3,
            },  
    overlay=True
    ).add_to(ft_grp4)

m.add_child(ft_grp1)
m.add_child(ft_grp2)
m.add_child(ft_grp3)
m.add_child(ft_grp4)

folium.LayerControl().add_to(m)
m

## Mapa 2 - Escolha uma rua e veja quais emoções foram atribuídas a ela

In [ ]:
nm_rua = raw_input ("Digite o nome de uma rua : ") 
# ideia: comboBox com as emoções da rua para escolher.

cursor.execute("""CREATE OR REPLACE VIEW rua_selec AS SELECT ruas_emoc_wgs.* FROM ruas_emoc_wgs WHERE nmvia = '%s'; """ %nm_rua)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l )) As properties
 FROM rua_selec As lg   ) As f )  As fc;""")
rua_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Geometria de ponto - Processo manual para simbolizar todas as emocoes
# ANIMACAO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts1 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE animacao >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts1 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua1 = json.dumps(cursor.fetchall())
conn.commit()

# ALEGRIA
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts2 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE alegria >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts2 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua2 = json.dumps(cursor.fetchall())
conn.commit()

# CANSACO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts3 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE cansaco >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts3 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua3 = json.dumps(cursor.fetchall())
conn.commit()

# BARULHO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts4 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE p_barulho >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts4 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua4 = json.dumps(cursor.fetchall())
conn.commit()

# ANSIEDADE
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts5 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE ansiedade >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts5 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua5 = json.dumps(cursor.fetchall())
conn.commit()

# Definindo o mapa e suas camadas
m = folium.Map(location = [-25.44,-49.24],
               zoom_start = 15,
               tiles = 'Cartodb Positron')

ruas_selec_emoc = folium.GeoJson(
    rua_geojson[2:len(rua_geojson)-2],
    name = 'Via escolhida',
    style_function = lambda feature: {
            'color': 'black',
            'weight': 2,
            },  
    overlay=True
    ).add_to(m)

# Grupos de camadas
ft_grp1 = folium.FeatureGroup(name = 'Positivas', show = False)
ft_grp2 = folium.FeatureGroup(name = 'Negativas', show = False)
grp1 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Animacao')
grp2 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Alegria')
grp3 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Cansaco')
grp4 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Barulho')
grp5 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Ansiedade')

# Camada dos pontos sobre as linhas
gjson_lnstopts_rua1 = json.loads(lnstopts_geojson_rua1[2:len(lnstopts_geojson_rua1)-2])
gjson_lnstopts_rua2 = json.loads(lnstopts_geojson_rua2[2:len(lnstopts_geojson_rua2)-2])
gjson_lnstopts_rua3 = json.loads(lnstopts_geojson_rua3[2:len(lnstopts_geojson_rua3)-2])
gjson_lnstopts_rua4 = json.loads(lnstopts_geojson_rua4[2:len(lnstopts_geojson_rua4)-2])
gjson_lnstopts_rua5 = json.loads(lnstopts_geojson_rua5[2:len(lnstopts_geojson_rua5)-2])

# --- ANIMACAO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'animacao'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url1 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua1['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url1,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp1.add_child(marker)

# --- ALEGRIA
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'alegria'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url2 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua2['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url2,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp2.add_child(marker)

# --- CANSACO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'cansaco'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url3 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua3['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url3,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp3.add_child(marker)

# --- BARULHO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'p_barulho'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url4 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua4['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url4,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp4.add_child(marker)

# --- ANSIEDADE
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'ansiedade'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url5 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua5['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url5,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp5.add_child(marker)

# Adicionando ao mapa os group layer
m.add_child(ft_grp1)
m.add_child(grp1)
m.add_child(grp2)
m.add_child(ft_grp2)
m.add_child(grp3)
m.add_child(grp4)
m.add_child(grp5)

folium.LayerControl(
    collapsed = False
).add_to(m)

m

## Mapa 3 - Faça uma rota e descubra quais emoções foram atribuídas as ruas em que a rota passou

In [ ]:
# Biblioteca para as rotas
!pip install openrouteservice ortools

In [ ]:
from geopy.geocoders import Nominatim
from openrouteservice import client

# estilo da rota
style = {'color': '#969696'}

# API do openroute services
api_key = '5b3ce3597851110001cf6248997aca5cd66d41c2aa3ad5f61948b94e'
clnt = client.Client(key=api_key) # Create client with api key

# Pede e geocodifica os dois endereços
geolocator = Nominatim (user_agent = 'camaragabriele@gmail.com')
endereco = raw_input ("Digite o primeiro endereco: ") 
location = geolocator.geocode (endereco)

endereco1 = raw_input ("Digite o segundo endereco: ") 
location1 = geolocator.geocode (endereco1)

# Coloca os pois pontos no mapa
m = folium.Map (location= [location.latitude, location.longitude],zoom_start=14, tiles = 'Cartodb Positron', min_zoom = 15)
folium. Marker (location=[location.latitude, location.longitude], popup=endereco,
                 icon=folium.Icon(color='blue',icon='star')).add_to (m)
folium. Marker (location=[location1.latitude, location1.longitude], popup=endereco,
                 icon=folium.Icon(color='green',icon='star')).add_to (m)

# Requisição da Rota                 
request_params = {'coordinates': [[location.longitude, location.latitude],
                                 [location1.longitude, location1.latitude]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}
route_normal = clnt.directions(**request_params)

# Adiciona a rota no mapa
rota = folium.features.GeoJson(data=route_normal,
                        name='Rota entre os pontos',
                        style_function=lambda x: style,
                        overlay=True).add_to(m)                 

dmb_gjson = json.loads(json.dumps(route_normal))

cursor.execute("""CREATE TABLE rota(id integer primary key); SELECT AddGeometryColumn ('public','rota','geom', 4326,'LINESTRING', 2);""")
conn.commit()

for feature in dmb_gjson['features']:
  geom = json.dumps(feature['geometry'])
  id = 1
  cursor.execute("""INSERT INTO rota (id, geom) VALUES (%s, ST_SetSRID(ST_GeomFromGeoJSON('%s'), 4326) )""" %(id, geom))
  conn.commit()

# Geometria de linha
cursor.execute("""CREATE OR REPLACE VIEW ruas_emoc_rotas AS SELECT ruas_emoc_wgs.* FROM rota, ruas_emoc_wgs WHERE ST_DWithin(ST_Transform(rota.geom, 31982), ST_Transform(ruas_emoc_wgs.geom_wgs, 31982), 50);""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM ruas_emoc_rotas As lg ) As f )  As fc;""")
rota_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Geometria de ponto - Processo manual para simbolizar todas as emocoes
# ANIMACAO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts1 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE animacao >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts1 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua1 = json.dumps(cursor.fetchall())
conn.commit()

# ALEGRIA
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts2 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE alegria >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts2 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua2 = json.dumps(cursor.fetchall())
conn.commit()

# CANSACO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts3 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE cansaco >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts3 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua3 = json.dumps(cursor.fetchall())
conn.commit()

# BARULHO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts4 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE p_barulho >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts4 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua4 = json.dumps(cursor.fetchall())
conn.commit()

# ANSIEDADE
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts5 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE ansiedade >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts5 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua5 = json.dumps(cursor.fetchall())
conn.commit()

# Camada das linhas
ruas_emoc_rota = folium.GeoJson(
    rota_emoc_geojson[2:len(rota_emoc_geojson)-2],
    name = 'Rua pertencente a rota',
    style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            },  
    overlay=True
    ).add_to(m)

# Grupos de camadas
ft_grp1 = folium.FeatureGroup(name = 'Positivas', show = False)
ft_grp2 = folium.FeatureGroup(name = 'Negativas', show = False)
grp1 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Animacao')
grp2 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Alegria')
grp3 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Cansaco')
grp4 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Barulho')
grp5 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Ansiedade')

# Camada dos pontos sobre as linhas
gjson_lnstopts_rua1 = json.loads(lnstopts_geojson_rua1[2:len(lnstopts_geojson_rua1)-2])
gjson_lnstopts_rua2 = json.loads(lnstopts_geojson_rua2[2:len(lnstopts_geojson_rua2)-2])
gjson_lnstopts_rua3 = json.loads(lnstopts_geojson_rua3[2:len(lnstopts_geojson_rua3)-2])
gjson_lnstopts_rua4 = json.loads(lnstopts_geojson_rua4[2:len(lnstopts_geojson_rua4)-2])
gjson_lnstopts_rua5 = json.loads(lnstopts_geojson_rua5[2:len(lnstopts_geojson_rua5)-2])

# --- ANIMACAO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'animacao'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url1 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua1['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url1,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp1.add_child(marker)

# --- ALEGRIA
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'alegria'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url2 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua2['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url2,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp2.add_child(marker)

# --- CANSACO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'cansaco'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url3 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua3['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url3,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp3.add_child(marker)

# --- BARULHO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'p_barulho'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url4 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua4['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url4,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp4.add_child(marker)

# --- ANSIEDADE
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'ansiedade'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url5 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua5['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url5,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp5.add_child(marker)

# Adicionando ao mapa os group layer
m.add_child(ft_grp1)
m.add_child(grp1)
m.add_child(grp2)
m.add_child(ft_grp2)
m.add_child(grp3)
m.add_child(grp4)
m.add_child(grp5)

folium.LayerControl(
).add_to(m)

m

# Rua Carlos Pradi, 163
# Rua Amoroso Costa, 100

Digite o primeiro endereco: Rua Carlos Pradi, 163
Digite o segundo endereco: Rua Amoroso Costa, 100


In [ ]:
cursor.execute(""" DROP TABLE IF EXISTS rota CASCADE; """)
conn.commit()

## Mapa 4 - Escolha um modal e veja quais emoções foram atribuídas a ele

In [ ]:
# Combox com os modais
# Funcao para fazer a lista do comboBox 
def lista_modal():
  modal = []
  cursor.execute("""SELECT modal_id, tipo FROM modais;""")
  fet_list = cursor.fetchall()
  conn.commit()
  for modal_id, tipo in fet_list:
    modal.append((tipo, modal_id))
  return modal

# Definindo a caixa de escolha
comboModal = widgets.Dropdown(
    options = lista_modal(),
    description='Modais:',
)
# Mostrando a caixa de escolha
display(comboModal)

RHJvcGRvd24oZGVzY3JpcHRpb249dSdNb2RhaXM6Jywgb3B0aW9ucz0oKHUnYSBwZScsIDEpLCAodSdhIHBlL29uaWJ1cycsIDIpLCAodSdiaWNpY2xldGEnLCAzKSwgKHUnY2Fycm8nLCA0KSzigKY=


In [ ]:
comboModal_esc = comboModal.value
cursor.execute("""CREATE OR REPLACE VIEW modal_selec AS SELECT trj_user_modal.* FROM trj_user_modal WHERE modal_id = %s; """ %comboModal_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT ogc_fid) As l)) As properties
FROM modal_selec As lg   ) As f )  As fc;""")
modal_geojson = json.dumps(cursor.fetchall())
conn.commit()
# cursor.close()
# conn.close()
m = folium.Map(location = [-25.44,-49.24],
               zoom_start = 15,
               tiles = 'Cartodb Positron')
modal_emoc = folium.GeoJson(
    modal_geojson[2:len(modal_geojson)-2],
    name = 'geojson'
    ).add_to(m)
m

## Mapa 5 - Escolha um usuário e veja quais emoções ele atribuíui nas ruas em que passou

In [ ]:
# Combox com os participantes
# Funcao para fazer a lista do comboBox 
def lista_users():
  users = []
  cursor.execute("""SELECT user_id, cod_user FROM usuario;""")
  fet_list = cursor.fetchall()
  conn.commit()
  for user_id, cod_user in fet_list:
    users.append((cod_user, user_id))
  return users

# Definindo a caixa de escolha
comboUser = widgets.Dropdown(
    options = lista_users(),
    description='Participantes:',
)
# Mostrando a caixa de escolha
display(comboUser)

In [ ]:
comboUser_esc = comboUser.value
cursor.execute("""CREATE OR REPLACE VIEW user_selec AS SELECT trj_user_modal.* FROM trj_user_modal WHERE user_id = %s; """ %comboUser_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT ogc_fid) As l)) As properties
FROM user_selec As lg   ) As f )  As fc;""")
user_geojson = json.dumps(cursor.fetchall())
conn.commit()
# cursor.close()
# conn.close()
m = folium.Map(location = [-25.44,-49.24],
               zoom_start = 15,
               tiles = 'Cartodb Positron')
modal_emoc = folium.GeoJson(
    user_geojson[2:len(user_geojson)-2],
    name = 'geojson'
    ).add_to(m)
m